In [ ]:
using Pkg; Pkg.activate(".")

In [ ]:
using JumpProcesses
using OrdinaryDiffEq
using DiffEqCallbacks
using Catalyst
using ModelingToolkit
using Random
using Distributions
using LogExpFunctions
using Plots
using StatsBase

# Lac operon

Simplified model of lac operon mostly inspired by [this textbook](https://www.math.uwaterloo.ca/~bingalls/MMSB/Notes.pdf#page=207.68).

In [ ]:
# extracellular lactose concentration as a time-dependent function
function lactose_conc(max_conc, t)
    return max_conc * logistic(10t - 60) * logistic(120 - 10t)
end

In [ ]:
rn = @reaction_network begin
    # lac repressor R binding switches the gene off, but allolactose A can bind to the repressor
    # and inactivate it, making it unable to repress the gene promoter
    mmr(A, σ_off, K_off), G_on --> G_off    # Gene deactivation rate
    σ_on, G_off --> G_on                    # Gene activation rate

    ρ_m, G_on --> G_on + M          # mRNA transcription 
    δ_m, M --> 0                    # mRNA degradation

    ρ_p, M --> M + P_Y + P_Z        # volume-dependent translation
    δ_p, P_Z --> 0                  # protein degradation for lacZ (β-galactosidase)
    δ_p, P_Y --> 0                  # protein degradation for lacY (β-galactoside permease)

    mm(lactose_conc(max_conc, t), k_L, K_ML), P_Y --> P_Y + L # lactose uptake catalysed by the β-galactoside permease
    
    mm(P_Z, k_A, K_MA), L --> A      # catalysed by β-galactosidase
    mm(P_Z, k_A, K_MA), L --> 0      # lactose metabolised into simpler sugars (catalysed by β-galactosidase)
    mm(P_Z, k_A, K_MA), A --> 0      # allolactose metabolised into simpler sugars (catalysed by β-galactosidase)
    δ_L, L --> 0                     # dilution of lactose
    δ_L, A --> 0                     # dilution of allolactose
end

In [ ]:
p = [ # Telegraph switching rates
     :σ_on => 10,
     :σ_off => 200,
     :K_off => 0.01,

      # mRNA production & degradation rates
     :ρ_m => 5,             
     :δ_m => 2.0,
    
      # Protein production & degradation rates
     :ρ_p => 5,             
     :δ_p => 0.5,
    
     # Lactose uptake & degradation rates
     :k_L => 10.0, 
     :K_ML => 0.5,
     :δ_L => 0.2,
    
     # Lactose metabolisation & allolactose production
     :k_A => 10,
     :K_MA => 10,
     :max_conc => 10.0
]

u0 = [ 
       :G_on => 0,
       :G_off => 1,
       :M => 0,
       :P_Y => 0,
       :P_Z => 0,
       :L => 0,
       :A => 0
    ]

tmax = 30.0
tspan = (0, tmax)
ts = 0:0.1:tmax;

In [ ]:
#ps = Dict(p)
#plot(ts, lactose_conc.(ts))
#plot(ts, mm.( lactose_conc.(ps[:max_conc], ts), ps[:k_L], ps[:K_ML]))

#plot(0:50 .* maximum(mm.(lactose_conc.(ts), ps[:k_L], ps[:K_ML])))

#plot(mmr.(0:50, ps[:σ_off], ps[:K_off]))
#plot(mm.(0:50, ps[:k_A], ps[:K_MA]), label="", xlabel="Number of P_Z molecules", ylabel="Rate")

## Deterministic rate equations

In [ ]:
oprob = ODEProblem(rn, u0, (0, tmax), p)
sol = solve(oprob, Tsit5(), saveat=ts);

In [ ]:
plot(sol.t, sol[:M], label="M")
plot!(sol.t, sol[:P_Y], label="P_Y")
plot!(sol.t, sol[:P_Z], label="P_Z")
plot!(sol.t, sol[:L], label="L", linewidth=3)
plot!(sol.t, sol[:A], label="A", linewidth=3)

## SSA trajectories

In [ ]:
jinp = JumpInputs(rn, u0, tspan, p)
jprob = JumpProblem(jinp, Direct(), save_positions=(false, false));

In [ ]:
jprob = remake(jprob, p = p)
@time sol = solve(jprob, Tsit5(), saveat=ts)
plot(sol.t, sol[:M], label="M")
plot!(sol.t, sol[:P_Y], label="P_Y")
plot!(sol.t, sol[:P_Z], label="P_Z")
plot!(sol.t, sol[:L], label="L", linewidth=3)
plot!(sol.t, sol[:A], label="A", linewidth=3)

In [ ]:
ensembleprob = EnsembleProblem(jprob)
@time ensemble_sol = solve(ensembleprob, Tsit5(), trajectories=100);

In [ ]:
@time ensemble_sol = solve(ensembleprob, Tsit5(), trajectories=1000);

Functions to plot means, variances and distributions (at certain time)

In [ ]:
# Compute the mean over the ensemble trajectories for the given species at times ts
function get_mean(sol::EnsembleSolution, sym::Symbol, ts::AbstractArray)
    ms = Vector{Float64}(undef, length(ts))
    for i in eachindex(ts)
        ms[i] = mean(sol[j](ts[i], idxs=sym) for j in eachindex(sol))
    end
    ms
end

# Compute the variance over the ensemble trajectories for the given species at times ts
function get_var(sol::EnsembleSolution, sym::Symbol, ts::AbstractArray)
    ms = Vector{Float64}(undef, length(ts))
    for i in eachindex(ts)
        ms[i] = var(sol[j](ts[i], idxs=sym) for j in eachindex(sol))
    end
    ms
end

In [ ]:
# Fit histogram to a given array of discrete counts
function fit_hist(ys::AbstractArray)
    max_y = maximum(ys)
    ws = fit(Histogram, ys, 0:max_y+1, closed=:left)
    ws = FrequencyWeights(ws.weights)
    0:max_y, ws
end

# Compute the distribution over the ensemble trajectories for the given species at a time point t
function get_dist(sol::EnsembleSolution, sym::Symbol, t::Real)
    ys = [sol[j](t, idxs=sym) for j in eachindex(sol)]
    fit_hist(ys)
end

# Plot the distribution over the ensemble trajectories for the given species at a time point t
function plot_dist(sol::EnsembleSolution, sym::Symbol, t::Real; kwargs...)
    _, ws = get_dist(sol, sym, t)
    xmax = length(ws)
    xs = 0:xmax
    ys = ws ./ ws.sum
    ys = vcat(ys, 0)
    plot(xs .- 0.5, zeros(xmax+1), linetype=:steppost, fillrange=ys; kwargs...)
end

In [ ]:
plot(ts, get_mean(ensemble_sol, :P_Z, ts), xlabel="Time", ylabel="Mean", label="β-galactosidase P_Z")
plot!(ts, get_mean(ensemble_sol, :L, ts), label="Lactose L")
#plot!(ts, get_mean(ensemble_sol, :A, ts), label="Allolactose A")

In [ ]:
plot_dist(ensemble_sol, :P_Y, 5.0, xlabel="Counts", ylabel="Probability", label="")

In [ ]:
pl1 = plot_dist(ensemble_sol, :P_Y, 5.0, xlabel="", ylabel="Probability", label="")
pl2 = plot_dist(ensemble_sol, :P_Y, 15.5, xlabel="", ylabel="", label="")
pl3 = plot_dist(ensemble_sol, :P_Y, 19.0, xlabel="Counts", ylabel="Probability", label="")
pl4 = plot_dist(ensemble_sol, :P_Y, 28.0, xlabel="Counts", ylabel="", label="")
plot(pl1, pl2, pl3, pl4)